In [54]:
from pyspark.sql import SparkSession

import os

spark = (SparkSession 
         .builder
         .config("spark.dynamicAllocation.maxExecutors","8")
         .config("spark.executor.memory", "4g")
         #.config("spark.hadoop.fs.s3a.block.size","1024M")
         #.config("spark.sql.files.maxPartitionBytes","1024M")
         #.config("spark.default.parallelism",32)
         #.config("spark.sql.shuffle.partitions",32)
         .enableHiveSupport()
         .getOrCreate()
        )

sc = spark.sparkContext

In [55]:
%cat /opt/hive/conf/hive-site.xml

<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>
<property>
    <name>hive.metastore.uris</name>
        <value>thrift://hive-metastore:9083</value>
    </property>
</configuration>


In [56]:
df = spark.read.format('csv').options(header='true', inferSchema='true', delimiter= ',').load("s3a://projet-arc-resil-poc/mad/fip_declarant_2021_sample.csv")
df.printSchema()

root
 |-- csdep: integer (nullable = true)
 |-- csdir: integer (nullable = true)
 |-- csdior: integer (nullable = true)
 |-- kccof: integer (nullable = true)
 |-- kccos: integer (nullable = true)
 |-- clcof: string (nullable = true)
 |-- datrfi: integer (nullable = true)
 |-- cidom4: string (nullable = true)
 |-- zvers: string (nullable = true)
 |-- ndcod: long (nullable = true)
 |-- ndcokd: integer (nullable = true)
 |-- cccoec: integer (nullable = true)
 |-- datrco: integer (nullable = true)
 |-- cidoco: string (nullable = true)
 |-- zverco: string (nullable = true)
 |-- dacoad: string (nullable = true)
 |-- dacomd: string (nullable = true)
 |-- dacojd: string (nullable = true)
 |-- cidetb: string (nullable = true)
 |-- cidetc: string (nullable = true)
 |-- cideek: string (nullable = true)
 |-- cideec: string (nullable = true)
 |-- cideea: string (nullable = true)
 |-- cideef: string (nullable = true)
 |-- cideer: string (nullable = true)
 |-- cideen: string (nullable = true)
 |-- ci

In [57]:
df.write.mode("overwrite").parquet("s3a://projet-arc-resil-poc/mad/parquet/fip_declarant_2021_sample")

In [58]:
spark.read.parquet("s3a://projet-arc-resil-poc/mad/parquet/fip_declarant_2021_sample").createOrReplaceTempView("fip_declarant_2021_sample")

In [59]:
spark.sql("SELECT csdep, datrfi from fip_declarant_2021_sample").show(1)

+-----+--------+
|csdep|  datrfi|
+-----+--------+
|   97|20210310|
+-----+--------+
only showing top 1 row



In [62]:
def check_array(dataType):
    if dataType.typeName()=="array":
        if "Double" in str(dataType):
            return "array<double>"
        elif "String" in str(dataType):
            return "array<string>"
        elif "Date" in str(dataType):
            return "array<date>"
        elif "Long" in str(dataType):
            return "array<long>"
        elif "Timestamp" in str(dataType):
            return "array<timestamp>"
        return str(dataType)+"oups"
    else:
        return dataType.typeName()
    return "array"
columns=",".join([f.name +" "+ check_array(f.dataType) for f in df.schema.fields if f.name !="h"])
createTable="CREATE EXTERNAL TABLE IF NOT EXISTS poc.fip_declarant_2021_sample ("+columns+") STORED AS PARQUET LOCATION 's3a://projet-arc-resil-poc/mad/parquet/fip_declarant_2021_sample';"
print(createTable)
spark.sql(createTable)

CREATE EXTERNAL TABLE IF NOT EXISTS poc.fip_declarant_2021_sample (csdep integer,csdir integer,csdior integer,kccof integer,kccos integer,clcof string,datrfi integer,cidom4 string,zvers string,ndcod long,ndcokd integer,cccoec integer,datrco integer,cidoco string,zverco string,dacoad string,dacomd string,dacojd string,cidetb string,cidetc string,cideek string,cideec string,cideea string,cideef string,cideer string,cideen string,cideed string,cideey string,cideee string,cideem string,cideeo string,cideeg string,cideet string,cicoef string,dacoef string,dmcoei string,dacoei string,cncoip string,cideci string,cifofc string,zoxyzd string,dacodj string,dacodm string,dacoda string,zoxyzp string,dacopj string,dacopm string,dacopa string,csdiam string,kccofm string,kccosm string,nscom string,cncofm string,cicokc string,ciifc5 string,cicoe2 string,cicot2 string,cicoe3 string,cicot3 string,cicoe4 string,cicot4 string,ciconv string,ciifse string,ciedsr string,cicosb string,cicopi string,cidea2 str

DataFrame[]

In [39]:
pip install trino -q

Note: you may need to restart the kernel to use updated packages.


In [40]:
import trino
conn = trino.dbapi.connect(
    host='projet-arc-resil-poc-428858.kub.sspcloud.fr',
    port=443,
    user='projet-arc-resil-poc',
    catalog='system',
    schema='schema',
    http_scheme='https',
    auth=trino.auth.BasicAuthentication("projet-arc-resil-poc", "resilTrino")
)
cur = conn.cursor()
cur.execute('SHOW CATALOGS')
rows = cur.fetchall()
print(rows)

[['hive1'], ['iceberg1'], ['postgresql2'], ['system'], ['tpcds'], ['tpch']]


In [41]:
cur.execute('SHOW SCHEMAS IN hive1')
rows = cur.fetchall()
print(rows)

[['default'], ['information_schema'], ['poc']]


In [63]:
cur.execute('SHOW tables IN hive1.poc')
rows = cur.fetchall()
print(rows)

[['datapump_table_2'], ['fip_declarant_2021_sample']]


In [43]:
%%time
cur.execute('SELECT count(*) from hive1.poc.datapump_table_2 where h=1')
rows = cur.fetchall()
print(rows)

[[9176]]
CPU times: user 33.4 ms, sys: 0 ns, total: 33.4 ms
Wall time: 723 ms
